In [ ]:
import boto3
import pandas as pd
import os
from io import StringIO
pd.set_option("display.max_columns", 50)
from matplotlib import pyplot as plt

In [ ]:
aws_access_key_id = os.getenv("AWS_ACCESS_KEY")
aws_secret_key = os.getenv("AWS_SECRET_KEY")

In [ ]:
def read_csv_from_s3(bucket: str, path: str, filename: str) -> pd.DataFrame:
    """Downloads a csv file from an S3 bucket.

    Parameters
     ----------
    bucket : str 
        The bucket where the files at.

    path : str
         The folders to the file.

    filename : str
        Name of the file.

    Returns
    ------
    pd.DataFrame
        A DataFrame of the downloaded file.
    """

    s3 = boto3.client("s3", aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_key) 

    full_path = f"{path}{filename}"

    object = s3.get_object(Bucket=bucket, Key=full_path)

    object = object["Body"].read().decode("utf-8")
    output_df = pd.read_csv(StringIO(object))

    return output_df

In [ ]:
s3 = boto3.client("s3", aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_key) 
bucket = "cubix-taxi-data-tm"

community_areas_path = "transformed_data/community_areas/"
payment_type_path = "transformed_data/payment_type/"
company_path = "transformed_data/company/"
weather_path = "transformed_data/weather/"
date_path = "transformed_data/date/"
taxi_trips_path = "transformed_data/taxi_trips/"

payment_type = read_csv_from_s3(bucket=bucket, path=payment_type_path, filename="Payment_type_master.csv")
company = read_csv_from_s3(bucket=bucket, path=company_path, filename="Company_master.csv")
date = read_csv_from_s3(bucket=bucket, path=date_path, filename="date_dimension.csv")
community_areas = read_csv_from_s3(bucket=bucket, path=community_areas_path, filename="community_areas_master.csv")

In [ ]:
trips_list = []
weather_list = []

In [ ]:
# TAXI TRIP DATA transform and loading
for file in s3.list_objects(Bucket=bucket, Prefix=taxi_trips_path)["Contents"]:
    taxi_trip_key = file["Key"]

    if taxi_trip_key.split("/")[-1].strip() != "":
        if taxi_trip_key.split(".")[1] == "csv":

            filename = taxi_trip_key.split("/")[-1]
            trip = read_csv_from_s3(bucket, taxi_trips_path, filename)
            trips_list.append(trip)
            print(f"{filename} has been added")

In [ ]:
trips = pd.concat(trips_list, ignore_index=True)

In [ ]:
for file in s3.list_objects(Bucket=bucket, Prefix=weather_path)["Contents"]:
    weather_key = file["Key"]

    if weather_key.split("/")[-1].strip() != "":
        if weather_key.split(".")[1] == "csv":

            filename = weather_key.split("/")[-1]
            weather_daily = read_csv_from_s3(bucket, weather_path, filename)
            weather_list.append(weather_daily)
            print(f"{filename} has been added")



In [ ]:
weather = pd.concat(weather_list, ignore_index=True)

In [ ]:
trips_full = pd.merge(trips, weather, left_on="datetime_for_weather", right_on="date_time", how="inner")
trips_full = trips_full.drop(columns=["date_time"])

In [ ]:
trips_full = pd.merge(trips_full, company, left_on="company_id", right_on="company_id", how="inner")
trips_full = trips_full.drop(columns=["company_id"])

In [ ]:
trips_full = pd.merge(trips_full, payment_type, left_on="payment_type_id", right_on="payment_type_id", how="inner")
trips_full = trips_full.drop(columns=["payment_type_id"])

In [ ]:
trips_full = pd.merge(trips_full, community_areas, left_on="pickup_community_area_id", right_on="area_code", how="inner")
trips_full = trips_full.drop(columns=["pickup_community_area_id"])
trips_full.rename(columns={"community_name": "pickup_community_area_id"})

In [ ]:
trips_full = pd.merge(trips_full, community_areas, left_on="dropoff_community_area_id", right_on="area_code", how="inner")
trips_full = trips_full.drop(columns=["dropoff_community_area_id"])
trips_full.rename(columns={"community_name": "dropoff_community_area_id"})

In [ ]:
date["Date"] = pd.to_datetime(date["Date"])
trips_full["trip_start_timestamp"] = pd.to_datetime(trips_full["trip_start_timestamp"])

trips_full["trip_start_date"] = trips_full["trip_start_timestamp"].dt.date
trips_full["trip_start_date"] = pd.to_datetime(trips_full["trip_start_date"])

In [ ]:
trips_full = pd.merge(trips_full, date, left_on="trip_start_timestamp", right_on="Date", how="inner")

In [ ]:
trips_full

In [ ]:
trips_per_day_df = trips_full.groupby("day_of_week", as_index=False)["trip_id"].count()

In [ ]:
trips_per_day_df

In [ ]:
days_nr = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}

In [ ]:
trips_per_day_df["day_names"] = trips_per_day_df["day_of_week"].replace(days_nr)

In [ ]:
trips_per_day_df

In [ ]:
plt.bar(trips_per_day_df["day_names"], trips_per_day_df["trip_id"])
plt.tight_layout()
plt.ylabel("Number of Trips")
plt.title("Number of Taxi trips per day")

plt.tight_layout()


In [ ]:
temp_per_day_df = trips_full.groupby("day", as_index=False)["temperature"].mean()
temp_per_day_df

In [ ]:
trips_per_day_df = trips_full.groupby("day", as_index=False)["trip_id"].count()
trips_per_day_df

In [ ]:
fig, ax1 = plt.subplots()

ax1.bar(trips_per_day_df["day"], trips_per_day_df["trip_id"])
ax1.set_ylabel("Number of Trips")
ax1.tick_params(axis="y")
ax1.set_xlabel("Day of July")

ax2 = ax1.twinx()
ax2.plot(temp_per_day_df["day"],temp_per_day_df["temperature"], color="red", marker="o")
ax2.set_ylabel("Mean Temperature [°C]", color="red")
ax2.tick_params(axis="y", labelcolor="red")
ax2.set_xticks(range(5, 16, 1))

plt.title("Mean temperature and number of trips for each day")
fig.tight_layout()

plt.show()

In [ ]:
wind_per_day_df = trips_full.groupby("day", as_index=False)["wind_speed"].mean()
wind_per_day_df

In [ ]:
fig, ax1 = plt.subplots()

ax1.bar(trips_per_day_df["day"], trips_per_day_df["trip_id"])
ax1.set_ylabel("Number of Trips")
ax1.tick_params(axis="y")
ax1.set_xlabel("Day of July")

ax2 = ax1.twinx()
ax2.plot(wind_per_day_df["day"],wind_per_day_df["wind_speed"], color="red", marker="o")
ax2.set_ylabel("Mean wind-speed [m/s]", color="red")
ax2.tick_params(axis="y", labelcolor="red")
ax2.set_xticks(range(5, 16, 1))

plt.title("Mean wind-speed and number of trips for each day")
fig.tight_layout()

plt.show()

In [ ]:
rain_per_day_df = trips_full.groupby("day", as_index=False)["rain"].mean()
rain_per_day_df

In [ ]:
fig, ax1 = plt.subplots()

ax1.bar(trips_per_day_df["day"], trips_per_day_df["trip_id"])
ax1.set_ylabel("Number of Trips")
ax1.tick_params(axis="y")
ax1.set_xlabel("Day of July")

ax2 = ax1.twinx()
ax2.plot(rain_per_day_df["day"],rain_per_day_df["rain"], color="red", marker="o")
ax2.set_ylabel("Rain perception", color="red")
ax2.tick_params(axis="y", labelcolor="red")
ax2.set_xticks(range(5, 16, 1))

plt.title("Rain perception and number of trips for each day")
fig.tight_layout()

plt.show()

In [ ]:
pay_per_day_df = trips_full.groupby("payment_type", as_index=False)["trip_id"].count()
pay_per_day_df

In [ ]:
plt.bar(pay_per_day_df["payment_type"], pay_per_day_df["trip_id"])
plt.tight_layout()
plt.ylabel("Number of Trips")
plt.title("Number of Taxi trips per payment-type")

plt.tight_layout()